To speed the execution procces, Colaboraty has an option to use GPU. This option is selected by default.
First of all we need to install two packages to show better the options of UFOD.

In [4]:
!pip3 install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import os

You should consider upgrading via the 'pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Workflow of the UFOD framework

The workflow of UFOD, depicted in the following image, captures all the necessary steps to train several object detection models and select the best one. Such a workflow can be summarised as follows. First of all, the user selects the dataset of images and some configuration parameters (mainly, the algorithms that will be trained and the frameworks or libraries that provide them). Subsequently, UFOD splits the dataset into a training set and a testing set. The training set is employed to construct several object detection models, and the best of those models is selected based on their performance on the testing set. The output of the framework is the best model, and an application, in the form of a Jupyter notebook, to employ such a model. Apart from the first step --- that is, the selection of the models to be trained --- the rest of the process is conducted automatically by UFOD without any user intervention. A more detailed explanation of the main features of this framework can be found in our [draft paper](https://github.com/ManuGar/UFOD/blob/master/draft.pdf).

<img src="https://github.com/ManuGar/UFOD/raw/master/images/DiagramUFOD.png" width="900" >

# How to use the framework

In case that UFOD is not installed in your system, the first task consists in installing using the github repositorie of the framework.

Clonando en 'UFOD'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 1211 (delta 132), reused 118 (delta 64), pack-reused 1018
Recibiendo objetos: 100% (1211/1211), 4.43 MiB | 6.17 MiB/s, listo.
Resolviendo deltas: 100% (803/803), listo.


To begin, we have to import all the classes that we will need to be able to use our framework.

In [10]:
import UFOD.taskLauncher
# import trainModel
import UFOD.factoryModel

### Configuring the dataset path

The dataset needs to be annotated using the Pascal VOC format. An application that produces such a format is [LabelImg](https://github.com/tzutalin/labelImg). The folder containing the dataset must contain two folders:
- JPEGImages folder: containing the images. 
- Annotations folder: containing the annotations using the Pascal format. 

This dataset will be later split. If the user wants to provide a explicit split, the organisation must be as follows:
- train
  - JPEGImages: folder containing the images of the training set. 
  - Annotations: folder containing the annotations of the training set. 
- test
  - JPEGImages: folder containing the images of the testing set. 
  - Annotations: folder containing the annotations of the testing set.
 

A dataset with this organization is provided in the [fruits folder](https://github.com/ManuGar/UFOD/tree/master/fruit) or [kangaro folder](https://www.dropbox.com/s/q19cl3agw8qt8e5/kangaroo.zip).


In [11]:
!wget "https://www.dropbox.com/s/q19cl3agw8qt8e5/kangaroo.zip?dl=0" -O kangaroo.zip
!unzip kangaroo.zip

--2019-12-11 10:50:11--  https://www.dropbox.com/s/q19cl3agw8qt8e5/kangaroo.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.1, 2620:100:6024:1::a27d:4401
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.1]:443... conectado.
Petición HTTP enviada, esperando respuesta... 301 Moved Permanently
Ubicación: /s/raw/q19cl3agw8qt8e5/kangaroo.zip [siguiente]
--2019-12-11 10:50:11--  https://www.dropbox.com/s/raw/q19cl3agw8qt8e5/kangaroo.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://ucd6253bf96f4de44aedc3903eda.dl.dropboxusercontent.com/cd/0/inline/AuBLLk2tR-mMxFsLRisQfc7UNIC7pMrNIU541VJ5pyhiM8UoGacmqx3506XrR3g69T5CeF14B9qBUIYtlVT-hkhmSHKK1jXxF1sj-g-87ujx_ihwiM17nObXP4nYmkVWi5s/file# [siguiente]
--2019-12-11 10:50:12--  https://ucd6253bf96f4de44aedc3903eda.dl.dropboxusercontent.com/cd/0/inline/AuBLLk2tR-mMxFsLRisQfc7UNIC7pMrNIU541VJ5pyhiM8UoGacmqx3506XrR3g69T5CeF14B9qBUIYtlVT-hkhmSHK

  inflating: kangaroo/dataset/00153.xml  
  inflating: kangaroo/dataset/00154.xml  
  inflating: kangaroo/dataset/00155.xml  
  inflating: kangaroo/dataset/00156.xml  
  inflating: kangaroo/dataset/00157.xml  
  inflating: kangaroo/dataset/00158.xml  
  inflating: kangaroo/dataset/00159.xml  
  inflating: kangaroo/dataset/00161.xml  
  inflating: kangaroo/dataset/00162.xml  
  inflating: kangaroo/dataset/00163.xml  
  inflating: kangaroo/dataset/00164.xml  
  inflating: kangaroo/dataset/00166.xml  
  inflating: kangaroo/dataset/00167.xml  
  inflating: kangaroo/dataset/00168.xml  
  inflating: kangaroo/dataset/00169.xml  
  inflating: kangaroo/dataset/00170.xml  
  inflating: kangaroo/dataset/00171.xml  
  inflating: kangaroo/dataset/00172.xml  
  inflating: kangaroo/dataset/00173.xml  
  inflating: kangaroo/dataset/00174.xml  
  inflating: kangaroo/dataset/00175.xml  
  inflating: kangaroo/dataset/00176.xml  
  inflating: kangaroo/dataset/00177.xml  
  inflating: kangaroo/dataset/0017

  inflating: kangaroo/JPEGImages/00013.jpg  
  inflating: kangaroo/JPEGImages/00014.jpg  
  inflating: kangaroo/JPEGImages/00016.jpg  
  inflating: kangaroo/JPEGImages/00017.jpg  
  inflating: kangaroo/JPEGImages/00018.jpg  
  inflating: kangaroo/JPEGImages/00019.jpg  
  inflating: kangaroo/JPEGImages/00020.jpg  
  inflating: kangaroo/JPEGImages/00021.jpg  
  inflating: kangaroo/JPEGImages/00022.jpg  
  inflating: kangaroo/JPEGImages/00023.jpg  
  inflating: kangaroo/JPEGImages/00024.jpg  
  inflating: kangaroo/JPEGImages/00025.jpg  
  inflating: kangaroo/JPEGImages/00026.jpg  
  inflating: kangaroo/JPEGImages/00027.jpg  
  inflating: kangaroo/JPEGImages/00028.jpg  
  inflating: kangaroo/JPEGImages/00029.jpg  
  inflating: kangaroo/JPEGImages/00030.jpg  
  inflating: kangaroo/JPEGImages/00031.jpg  
  inflating: kangaroo/JPEGImages/00032.jpg  
  inflating: kangaroo/JPEGImages/00033.jpg  
  inflating: kangaroo/JPEGImages/00034.jpg  
  inflating: kangaroo/JPEGImages/00036.jpg  
  inflatin

### Preparing the environment

UFOD training process is configured by means a list of parameters, that must contain the following information:

- "dataset": path to the dataset of images. 
- "dataset_name": a name to the dataset.
- "exec": the execution mode. There are two execution modes:
  - "local". The training process is run locally. In this option it is necessary to provide the number of available gpus with the "ngpus" parameter. 
  - "slurm". The training process is run in a cluster with SLURM. There are several parameters to configure in this option, such as the execution time ("time" parameter), the partition ("partition" parameter), the gres ("gres" parameter), the memory ("mem" parameter), or the number of gpus  ("ngpus" parameter).
- "frameworks": the list of models to train. Currently, the following options are available:
  - ["Mxnet","ssdVgg16"]: SSD with the VGG16 backbone of the MXNet framework. 
  - ["Mxnet","ssdVgg16_512"]: SSD with the VGG16 backbone and image size of 512 of the MXNet framework. 
  - ["Mxnet","ssdResnet"]: SSD with the Resnet backbone of the MXNet framework.
  - ["Mxnet","ssdMobilenet"]: SSD with the mobilenet backbone of the MXNet framework.
  - ["Rcnn","mask-rcnn"]: Mask-RCNN algorithm using a Keras library. 
  - ["Darknet","yolo"]: YOLO with the Darknet framework.
  - ["Darknet","tinyYolo"]: TinyYOLO with the Darknet framework.
  - ["Retinanet","retinanet"]: RetinaNet algorithm using a Keras library. 
  - ["Tensorflow", "ssdInception"]: SSD with the inception backbone using the Tensorflow framework.
  - ["Tensorflow", "fasterRcnnResnet"]: Faster RCNN with the inception backbone using the Tensorflow framework.
  - ["Tensorflow", "rfcnResnet"]: RFCN with the inception backbone using the Tensorflow framework.
  - ["Tensorflow", "maskRcnnInception"]: Mask RCNN with the inception backbone using the Tensorflow framework.

In [12]:
dataset_path = widgets.Text(
       value='./fruit',
       description='Dataset path', )
dataset_name = widgets.Text(
       value='fruit',
       description='Dataset name', )
box = widgets.VBox([dataset_path, dataset_name])
box

In [16]:
MxnetVgg16 = widgets.Checkbox(
           description='Mxnet-SsdVgg16',value=True)
MxnetVgg16_512 = widgets.Checkbox(
           description='Mxnet-SsdVgg16_512',value=True)
MxnetResnet = widgets.Checkbox(
           description='Mxnet-SsdResnet',value=True)
MxnetMobilenet = widgets.Checkbox(
           description='Mxnet-SsdMobilenet',value=True)
Mrcnn = widgets.Checkbox(
           description='Mask-RCNN',value=True)
Retinanet = widgets.Checkbox(
           description='Retinanet',value=True)
DarknetYolo = widgets.Checkbox(
           description='Darknet-Yolo v3',value=True)
DarknetTinyYolo = widgets.Checkbox(
           description='Darknet-Tiny Yolo v3',value=True)
TensorflowSsdInception = widgets.Checkbox(
           description='Tensorflow-SsdInception',value=True)
TensorflowfasterRcnnResnet = widgets.Checkbox(
           description='Tensorflow-FasterRCNNRestnet',value=True)
TensorflowRfcnResnet = widgets.Checkbox(
           description='Tensorflow-RFCNResnet',value=True)
TensorflowMaskRcnnInception = widgets.Checkbox(
           description='Tensorflow-MaskRCNNInception',value=True)


box1 = widgets.VBox([MxnetVgg16,MxnetVgg16_512,MxnetResnet,MxnetMobilenet])
box2 = widgets.VBox([Mrcnn,Retinanet,DarknetYolo,DarknetTinyYolo])
box3 = widgets.VBox([TensorflowSsdInception,
                    TensorflowfasterRcnnResnet,TensorflowRfcnResnet,TensorflowMaskRcnnInception])

frameworks=[]
widgets.HBox([box1, box2,box3])


In [17]:
if(MxnetVgg16.value):
    frameworks.append(["Mxnet","ssdVgg16"])
if(MxnetVgg16_512.value):
    frameworks.append(["Mxnet","ssdVgg16_512"])
if(MxnetResnet.value):
    frameworks.append(["Mxnet","ssdResnet"])
if(MxnetMobilenet.value):
    frameworks.append(["Mxnet","ssdMobilenet"])
if(Mrcnn.value):
    frameworks.append(["Rcnn","mask-rcnn"])
if(Retinanet.value):
    frameworks.append(["Retinanet","retinanet"])
if(DarknetYolo.value):
    frameworks.append(["Darknet","yolo"])
if(DarknetTinyYolo.value):
    frameworks.append(["Darknet","tinyYolo"])
if(TensorflowSsdInception.value):
    frameworks.append(["Tensorflow","ssdInception"])
if(TensorflowfasterRcnnResnet.value):
    frameworks.append(["Tensorflow","fasterRcnnResnet"])
if(TensorflowMaskRcnnInception.value):
    frameworks.append(["Tensorflow","rfcnResnet"])
if(TensorflowMaskRcnnInception.value):
    frameworks.append(["Tensorflow","maskRcnnInception"])



type = widgets.Dropdown(
    options=['local', 'slurm'],
    value='local',
    description='Execution type:',
    disabled=False,
)

type

Dropdown(description='Execution type:', options=('local', 'slurm'), value='local')

In [24]:
n_gpus = widgets.BoundedIntText(
        value=1,
        min=1,
        max=15,
        step=1,
        description='Number GPU\'s:',
        disabled=False
    )
if type.value=="slurm":
    time = widgets.Text(value='72:00:00',description='Time (HH:MM:SS)', )
    partition = widgets.Text(value='gpu',description='Partition', )
    gres = widgets.Text(value='gpu:kepler:2',description='Gres', )
    mem = widgets.Text(value='56GB',description='Memmory', )
    box1 =widgets.VBox([time, partition,gres])
    box2 =widgets.VBox([mem, n_gpus])
    newbox = widgets.HBox([box1,box2])
elif type.value =="local":
    newbox = widgets.HBox([n_gpus])

newbox

### How to launch the training process

Once the user has prepared the dataset and fixed the training options in the previous parameters, the training process is launched using the following command:


In [28]:

for fram, mod in frameworks:
    file_name = os.path.join("./UFOD","scripts", "train_" + fram + "_" + mod + "_" + dataset_name.value + ".sh")
    f = open(file_name, "w")
    f.write("#!/bin/sh\n")
    if type.value=="slurm":
        if (fram == "Mxnet"):
            f.write("source configs_slurm/mxnet.sh\n")
        if (fram == "Rcnn"):
            f.write("source configs_slurm/maskrcnn.sh\n")
        if (fram == "Retinanet"):
            f.write("source configs_slurm/retinanet.sh\n")
        if (fram == "Tensorflow"):
            f.write("source configs_slurm/tensorflow.sh\n")
        if (fram == "Darknet"):
            f.write("source configs_slurm/yolo.sh\n")                
    elif type =="local":
        if (fram == "Mxnet"):
            f.write("source configs_local/mxnet.sh\n")
        if (fram == "Rcnn"):
            f.write("source configs_local/maskrcnn.sh\n")
        if (fram == "Retinanet"):
            f.write("source configs_local/retinanet.sh\n")
        if (fram == "Tensorflow"):
            f.write("source configs_local/tensorflow.sh\n")
        if (fram == "Darknet"):
            f.write("source configs_local/yolo.sh\n")
    f.write("python3 trainModel.py -f " + fram + " -m " + mod + " -d " + dataset_path.value + " -dn " + dataset_name.value + " -ng " + str(n_gpus.value))
    if type == "slurm":
        os.system("sbatch -p " + partition + " --gres=" + gres + " --time=" + time+ " --mem=" + mem + " " + file_name )
    elif type =="local":
        os.system("bash " + file_name)



In [0]:
# model = factoryModel.createModel(framework, modelText, dataset, dataset_name)
# model.split(0.75)
# model.transform()
# model.createModel()
# model.train(conf[framework], ngpus)

In [35]:
os.system("bash UFOD/scripts/train_Darknet_tinyYolo_fruit.sh")

512

In [32]:
!ps

  PID TTY          TIME CMD
10874 pts/1    00:00:00 sh
10875 pts/1    00:00:00 ps
